**Pycaret 실습해보기**
* anaconda 설치 후 가상환경에서 실습 진행
* pycaret install 

```
$ pip install pycaret
```

In [1]:
#pycaret 
from pycaret.regression import *

In [2]:
#Loading Dataset
import seaborn as sns
dataset = sns.load_dataset('titanic')
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
#Train_Test split
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size = 0.3)

print(train_data.shape)
print(test_data.shape)

(623, 15)
(268, 15)


In [4]:
#Data preprocessing & Create Pipeline
ixi_model = setup(session_id=1, data=train_data, target='age', test_data= test_data, use_gpu=True, n_jobs=10, remove_perfect_collinearity = True)

# 각 컬럼에 Type을 표시 -> 맞을 경우 Enter, 지금까지의 내용에 대한 요약
# 선택한 옵션은 초록색으로 표시

,Description,Value
0,session_id,1
1,Target,age
2,Original Data,"(623, 15)"
3,Missing Values,True
4,Numeric Features,1
5,Categorical Features,13
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(493, 35)"


In [5]:
models()  # 사용 가능한 모델 

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


모델 적용해보기

In [6]:
# Model Conpare
pycaret_regression_models = compare_models(n_select = 2, sort = "MAE", include = ["lr","lasso","ridge","en","lar","llar","omp","rf"])

#좋은 성능을 보이는 부분을 노란색으로 표시

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,8.9730,129.1725,11.2533,0.3960,0.4181,0.4590,0.0350
lr,Linear Regression,9.0175,130.6590,11.3183,0.3887,0.4209,0.4493,0.0200
omp,Orthogonal Matching Pursuit,9.0977,136.4392,11.5672,0.3622,0.4191,0.4614,0.0160
rf,Random Forest Regressor,9.2391,147.4359,12.0222,0.3041,0.4216,0.4553,0.7690
lasso,Lasso Regression,9.6225,147.6235,12.0684,0.3108,0.5228,0.7853,0.0210
en,Elastic Net,10.6299,176.4625,13.2103,0.1763,0.6185,1.1066,0.0190
llar,Lasso Least Angle Regression,11.4944,216.2983,14.6444,-0.0128,0.6815,1.3541,0.0160
lar,Least Angle Regression,10349.0981,27855135310.3031,56501.7150,-128355698.0940,1.8908,436.4957,0.0430


In [7]:
pycaret_regression_models

#best model에 대한 정보

[Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
       normalize=False, random_state=1, solver='auto', tol=0.001),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=10, normalize=False)]

In [8]:
best_model = create_model('ridge')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,6.9879,76.3178,8.7360,0.5870,0.3469,0.3513
1,9.0627,125.0253,11.1815,0.3507,0.4711,0.5461
2,9.6237,130.3688,11.4179,0.1824,0.4162,0.3922
3,7.9712,99.0013,9.9499,0.5243,0.4471,0.5994
4,9.4993,136.1641,11.6689,0.3730,0.3739,0.3228
5,7.3047,93.3804,9.6634,0.4515,0.3721,0.4151
6,10.2577,204.8429,14.3123,0.3259,0.4461,0.5077
7,9.0223,129.4411,11.3772,0.3581,0.4362,0.5722
8,11.3692,181.7580,13.4818,0.2642,0.4956,0.4752


In [9]:
#tunning 
from sklearn.linear_model import *

#defualt_lasso_model = Lasso()
tuned_model = tune_model(best_model, n_iter= 100,optimize ="MAE")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,7.1529,77.1010,8.7807,0.5827,0.3643,0.3807
1,8.7433,117.5930,10.8440,0.3893,0.4640,0.5588
2,9.4757,126.6447,11.2537,0.2057,0.4007,0.3924
3,8.0751,99.1548,9.9577,0.5236,0.4836,0.7325
4,9.3009,134.1223,11.5811,0.3824,0.3634,0.3015
5,7.2472,95.4573,9.7702,0.4393,0.3889,0.4667
6,10.3710,206.5716,14.3726,0.3202,0.4632,0.5532
7,9.1042,131.8453,11.4824,0.3462,0.4544,0.6155
8,11.3000,182.2408,13.4997,0.2622,0.4461,0.4686


In [10]:
tuned_model

#파라미터 값이 변경된 것을 알 수 있음 

Ridge(alpha=5.07, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=1, solver='auto', tol=0.001)

In [11]:
#save best model

save_model(tuned_model, './best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='age',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='me...
                 ('dummy', Dummify(target='age')),
                 ('fix_perfect', Remove_100(target='age')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),


In [12]:
#Load model

load_model = load_model('./best_model')

Transformation Pipeline and Model Successfully Loaded


In [13]:
load_model['trained_model']

Ridge(alpha=5.07, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=1, solver='auto', tol=0.001)